In [ ]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.accuracy import Likelihood
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork
from dfg_rating.model.rating.controlled_trend_rating import ControlledTrendRating, ControlledRandomFunction

#specify rating error
from dfg_rating.model.rating.base_rating import RatingFunctionError
rating_error = RatingFunctionError(error='normal', loc=0, scale=50)

import networkx as nx
import pandas as pd
import numpy as np

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
numberLeagues = 5
networks = []
#np.empty(numberLeagues, dtype=network) 
for league in range(numberLeagues):
    print(league)
    network: BaseNetwork = factory.new_network(
        'multiple-round-robin',
        teams=18,
        days_between_rounds=7,
        seasons=10,
        league_teams=18,
        league_promotion=0,
        create=True,
        true_forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients = [-0.9,0.3], beta_parameter=0.006),
        true_rating=ControlledTrendRating(starting_point=ControlledRandomFunction(distribution='normal', loc=1000, scale=100),
                                     delta=ControlledRandomFunction(distribution='normal', loc=0, scale=3),
                                     trend=ControlledRandomFunction(distribution='normal', loc=0, scale=20/365),
                                     season_delta=ControlledRandomFunction(distribution='normal', loc=0, scale=10))
    )

networks.append(network)
#network.create_data()

In [ ]:
#network.print_data(schedule=True)

In [ ]:
#adding biased bookmaker forecast
network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0.0], beta_parameter=0.006, home_team_error=rating_error, away_team_error=rating_error),
    forecast_name='bookmaker_forecast',
    base_ranking='true_rating'
)

In [ ]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.10)
)
network.add_odds(
    bookmaker_name="bm",
    bookmaker=bookmaker,
    base_forecast='bookmaker_forecast'
)

In [ ]:
betting = FixedBetting(100)

In [ ]:
def aggregate_measures(list_of_matches, prefix=''):
    return {prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]), 
           prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]), 
           prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
           prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches])}

In [ ]:
result_list = []
#for c0 in np.arange(-1.0, 0.6, 0.1):
#    for c1 in np.arange(c0+0.2, 0.8, 0.1):
#        for beta in np.arange(0.004, 0.010, 0.002):
for c0 in np.arange(-1.0, -0.6, 0.1):
    for c1 in np.arange(c0+0.2, 0.0, 0.1):
        for beta in np.arange(0.004, 0.010, 0.002):
            
            print(f'Variables {c0}, {c1} and {beta}')
            all_matches =  []
            is_matches =  []
            oos_matches =  []
            
            for network in networks:
                rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
                betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
                likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            
                network.add_forecast(
                forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta, home_team_error=rating_error, away_team_error=rating_error),
                forecast_name='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta),
                base_ranking='true_rating'
                )
                
                network.add_bets(
                bettor_name='b',
                bookmaker='bm',
                betting=betting,
                base_forecast='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)
                )
            
                network.add_evaluation(rps, 'rps')
                network.add_evaluation(betting_returns, 'betting_returns')
                network.add_evaluation(likelihood, 'likelihood')
                #network.serialize_network('economic_network')
                
                all_matches.append([(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games()])
                is_matches.append([(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 4])
                oos_matches.append([(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 3])
                
            result_all = aggregate_measures(all_matches, 'all')
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos, **result_all
            }
            
            result_list.append(result_all_leagues)
            
                 
            
#network.export(forecasts = ['bookmaker_forecast', 'true_forecast', 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)],
#                      metrics = ['rps', 'betting_returns', 'loglikelihood'],
#                      odds=['bm'])
            
            

            

In [ ]:
df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")

In [ ]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])